In [5]:
import pandas
import os
import numpy as np


df = pandas.read_csv("1335.vdb.tab", sep="\t", header =None) 
#Récupère la liste de nombre, correspondant aux comptage des différents espèces en fonction des lieu de prélevement
#2=door_in_1 ; 3=door_in_2 ; 4=faucet_handle_1 ; 5=faucet_handle_2 ; 6=sink_floor_1 ; 7=sink_floor_2
#8=soap_dispenser_1 ; 9=stall_in_1 ; 10=toilet_floor_1 ; 11=toilet_floor_1 ; 12=toilet_flush_handle_1
#13=toilet_flush_handle_2 ; 14=toilet_seat_1 ; 15=toilet_seat_2
def getcountsfromrow(row):
    l_l = [ int(row[2]),
            int(row[3]),
            int(row[4]),
            int(row[5]),
            int(row[6]),
            int(row[7]),
            int(row[8]),
            int(row[9]),
            int(row[10]),
            int(row[11]),
            int(row[12]),
            int(row[13]),
            int(row[14]),
            int(row[15]) ]
    return l_l

"""
Code pour generer dictionnaire. 

Au prealable, avec cat 1335.vdb.tab | cut -f 2 | cut -d ";" -f 1 | uniq
OTUConTaxonomy"Bacteria(100)  => on a confirmé que ce sont toutes Bacteria
"""
def getDicoTaxo(df):
    dicotaxo = { "name" : "Bacteria", "count" : [0 for i in range(14)], "children" : []}
    for index,row in df.iterrows():
        leveltax = row[1].split(';')  # uniquement indexes >=1 m'interessent 
        if index > 0:
            COUNTS = getcountsfromrow(row)
            tmp5 = {}
            tmp4 = {}
            tmp3 = {}
            tmp2 = {}
            tmp1 = {}
            tmp5["name"] = leveltax[5]
            tmp4["name"] = leveltax[4]
            tmp3["name"] = leveltax[3]
            tmp2["name"] = leveltax[2]
            tmp1["name"] = leveltax[1]      
            tmp5["count"] =  getcountsfromrow(row)
            tmp5["children"] = []  # lowest level
            tmp4["count"] =  getcountsfromrow(row)
            tmp4["children"] = [tmp5]
            tmp3["count"] =  getcountsfromrow(row)
            tmp3["children"] = [tmp4]
            tmp2["count"] =  getcountsfromrow(row)
            tmp2["children"] = [tmp3]
            tmp1["count"] = getcountsfromrow(row)
            tmp1["children"] = [tmp2]
            tmpcounts = dicotaxo["count"]
            dicotaxo["count"] = np.add(tmpcounts, COUNTS)
            if leveltax[1] not in [m["name"] for m in dicotaxo["children"]] :
                dicotaxo["children"].append(tmp1)
            else:
                for m in dicotaxo["children"]:
                    if m["name"] == leveltax[1]:
                        tmpc = m["count"]
                        m["count"] = np.add(tmpc, COUNTS)
                        if leveltax[2] not in [n["name"] for n in m["children"]] :
                            m["children"].append(tmp2)
                        else:
                            for n in m["children"]:
                                if n["name"] == leveltax[2]:
                                    tmpc = n["count"]
                                    n["count"] = np.add(tmpc, COUNTS)
                                    if leveltax[3] not in [o["name"] for o in n["children"]]:
                                        n["children"].append(tmp3)
                                    else: 
                                        for o in n["children"]:
                                            if o["name"] == leveltax[3]:
                                                tmpc = o["count"]
                                                o["count"] = np.add(tmpc,COUNTS)
                                                if leveltax[4] not in [p["name"] for p in o["children"]]:
                                                    o["children"].append(tmp4)
                                                else:
                                                    for p in o["children"]:
                                                        if p["name"] == leveltax[4]:
                                                            tmpc = p["count"]
                                                            p["count"] = np.add(tmpc, COUNTS)
                                                            if leveltax[5] not in [q["name"] for q in p["children"]]:
                                                                p["children"].append(tmp5)
                                                            else:
                                                                for q in p["children"]:
                                                                    if q["name"] == leveltax[5]:
                                                                        tmpc = q["count"]
                                                                        q["count"] = np.add(tmpc, COUNTS)
                                                                        q["children"] = []
                                                                        # this is the lowest level                                        
    return dicotaxo


In [3]:
import requests
import pandas
import matplotlib.pyplot as plt
import numpy as np

df = pandas.read_csv("1335.vdb.tab", sep="\t", header =None) 
somme = 0
for index,row in tb.iterrows():
    
    if index >0 : 
        somme = int(row[6]) + somme
somme
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,toilet_floor_1,toilet_floor_2,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2
1,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,45,87,13,329,76,156
2,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,430,707,0,25,258,0
3,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,6,30,3,283,41,1059
4,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,236,251,9,51,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4101,Otu04101,Bacteria(100);Bacteroidetes(100);Cytophagia(10...,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4102,Otu04102,Bacteria(100);Proteobacteria(100);Alphaproteob...,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4103,Otu04103,Bacteria(100);Firmicutes(100);unclassified(100...,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4104,Otu04104,Bacteria(100);Acidobacteria(100);Acidobacteria...,0,0,0,0,0,1,0,0,0,0,0,0,0,0
